### 문제 2-4 : 여행 계획 분석기
- 출력 파서: StructuredOutputParser
- 문제
여행 후기나 계획 텍스트를 입력받아 여행지, 기간, 예산, 추천도(1-5점), 주요 활동 리스트를 구조화된 형태로 추출하는 시스템을 만드세요.
- 요구사항:
    - StructuredOutputParser와 ResponseSchema 사용
    - 5개의 필드를 정의하여 정보 추출
    - 자연어 텍스트에서 핵심 정보 파싱
- 예시 입력:
 "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
- 예시 출력 구조:
{
    "destination": "부산",
    "duration": "2박 3일",
    "budget": "30만원",
    "rating": "4",
    "activities": ["해운대 바다구경", "자갈치시장 회 먹기", "감천문화마을 구경"]
}

In [ ]:

from typing import List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# 1) 출력 스키마 정의 (필드 5개)
schemas: List[ResponseSchema] = [
    ResponseSchema(
        name="destination",
        description="여행지(도시/지역 한 단어 또는 짧은 구). 예: 부산, 제주, 강릉"
    ),
    ResponseSchema(
        name="duration",
        description="여행 기간 서술(예: '2박 3일', '3일', '주말 1박2일')"
    ),
    ResponseSchema(
        name="budget",
        description="총 예산/지출. 숫자+단위 형태 유지(예: '30만원', '45만 원', '120,000원')"
    ),
    ResponseSchema(
        name="rating",
        description="추천도 1~5점 중 정수만. 텍스트에 비율/별점이 있어도 1~5로 환산하여 숫자만 출력(예: '4')"
    ),
    ResponseSchema(
        name="activities",
        description="주요 활동 리스트. 간결한 구/동사구로 3~6개 내외. 예: ['해운대 바다구경','자갈치시장 회 먹기']"
    ),
]

parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = parser.get_format_instructions()

# 2) 프롬프트
prompt = PromptTemplate(
    template=(
        "아래 여행 후기/계획 텍스트에서 핵심 정보를 추출하라.\n"
        "- 한국어 입력을 처리한다.\n"
        "- 추천도는 1~5 사이 정수만 허용한다(소수/5점 만점 서술 → 정수로 근사).\n"
        "- 활동은 간결한 구로 정리하고, 중복/군더더기 표현은 제거한다.\n"
        "- 출력 형식은 반드시 아래 지시를 따르라.\n\n"
        "{format_instructions}\n\n"
        "입력 텍스트:\n{text}\n"
    ),
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions},
)

# 3) LLM 설정 (원하는 엔드포인트 사용 가능)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

def extract_trip_info(text: str) -> Dict[str, Any]:
    """자연어 여행 텍스트 → 구조화된 dict"""
    _prompt = prompt.format(text=text)
    resp = llm.invoke(_prompt).content
    data = parser.parse(resp)

    # --- (선택) 후처리 안전장치 ---
    # rating을 '1'~'5' 문자열로 강제
    try:
        r = int(str(data.get("rating", "")).strip())
        r = min(5, max(1, r))
        data["rating"] = str(r)
    except Exception:
        data["rating"] = None

    # activities 공백/중복 정리
    if isinstance(data.get("activities"), list):
        seen = set()
        cleaned = []
        for a in data["activities"]:
            s = str(a).strip()
            if s and s not in seen:
                cleaned.append(s)
                seen.add(s)
        data["activities"] = cleaned

    return data

# 4) 테스트
if __name__ == "__main__":
    sample = (
        "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 "
        "해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. "
        "정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
    )
    result = extract_trip_info(sample)
    print(result)